# Setup

In [0]:
storage_account_name = dbutils.secrets.get(scope="kv-scope", key="storage-account-name")
container_name = "data"
mount_point = "/mnt/data"
file_path = "farmers-protest-tweets-2021-2-4.json"
dbfs_file_path = f"/dbfs{mount_point}/{file_path}"

In [0]:
# Create this mount, if it does not already exist
if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
        mount_point = mount_point,
        extra_configs  = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net" : dbutils.secrets.get(scope="kv-scope", key="access-key")}
    )

In [0]:
# Install all relevant libraries with minimal output
%pip install -q memory_profiler py-spy emoji polars

In [0]:
from memory_profiler import profile
import importlib

# Challenge

## q1_memory

In [0]:
from src.q1_memory import q1_memory

In [0]:
result = q1_memory(dbfs_file_path)
result

## q1_time

In [0]:
from src.q1_time import q1_time

In [0]:
result = q1_time(dbfs_file_path)
result

## q2_memory

In [0]:
from src.q2_memory import q2_memory

In [0]:
result = q2_memory(dbfs_file_path)
result

## q2_time

In [0]:
from src.q2_time import q2_time

In [0]:
result = q2_time(dbfs_file_path)
result

## q3_memory

## q3_time